# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys

os.environ['MKL_NUM_THREADS'] = '1'

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('../raw_data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

In [11]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,18119983,18120172,18120301,18120717,18120821,18121351,18148533,18148596,18183625,18184178
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [14]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [15]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 2.84 s


-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [ ]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        self.data = prefilter_items(data, take_n_popular=5000)
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(
            self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data: pd.DataFrame):
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id',
                                          columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )
        user_item_matrix = user_item_matrix.astype(float)
        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def get_rec_similar_items(self, x):
        """Находит товар, похоий на x"""
        recs = self.model.similar_items(self.itemid_to_id[x], N=2)
        top_rec = recs[1][0]
        return self.id_to_itemid[top_rec]

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        popular_items = self.data.loc[self.data['user_id'] == user, :].groupby('item_id')['quantity'].sum().reset_index()
        popular_items.rename(columns={'quantity': 'n_sold'}, inplace=True)
        popular_items = popular_items.loc[popular_items['item_id'] != 999999]
        top_n_items = popular_items.sort_values('n_sold', ascending=False).head(N).item_id.tolist()
        similar_items = [self.get_rec_similar_items(item) for item in top_n_items]

        assert len(similar_items) == N, 'Количество рекомендаций != {}'.format(N)
        return similar_items

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        similar_users = self.model.similar_users(self.userid_to_id[user], N)
        res = []
        for similar_user in similar_users:
            popular = self.data.loc[self.data['user_id'] == similar_user[0], :].groupby('item_id')[
                'quantity'].sum().reset_index()
            popular.rename(columns={'quantity': 'n_sold'}, inplace=True)
            popular = popular.loc[popular['item_id'] != 999999]
            top_5 = popular.sort_values('n_sold', ascending=False).head(5).item_id.tolist()
            res.append(np.random.choice(top_5))

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

Проверка, что все работает

In [4]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [5]:
%%time
recommender = MainRecommender(data_train)


Wall time: 49.1 s


In [6]:
closest_items, top_n_items = recommender.get_similar_items_recommendation(user=2374)
closest_items

[9270263, 943892, 6633054, 15452032, 5995150]

In [7]:
top_n_items

[1059556, 15596933, 13213342, 13190673, 13213209]

In [8]:
item_features[item_features.item_id.isin(top_n_items)]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
33008,1059556,2172,DRUG GM,National,CANDY - PACKAGED,SEASONAL MISCELLANEOUS,1 OZ
82180,13190673,693,DRUG GM,National,CANDY - PACKAGED,CANDY BARS (SINGLES)(INCLUDING,
82305,13213209,1046,GROCERY,National,PWDR/CRYSTL DRNK MX,SOFT DRINK POWDER POUCHES,10 QT
82347,13213342,69,GROCERY,Private,PWDR/CRYSTL DRNK MX,SOFT DRINK POWDER POUCHES,1.9 OZ
87302,15596933,693,DRUG GM,National,CANDY - PACKAGED,CANDY BARS (MULTI PACK),4.5 OZ


In [9]:
item_features[item_features.item_id.isin(closest_items)]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
20106,943892,857,DRUG GM,National,CANDY - PACKAGED,CANDY BAGS-CHOCOCLATE,12.7 OZ
55313,5995150,1046,GROCERY,National,PWDR/CRYSTL DRNK MX,SOFT DRINK CANISTERS,2.53Z/10QT
58166,6633054,2172,DRUG GM,National,CANDY - PACKAGED,DIET CNTRL BARS NUTRITIONAL,3.5 OZ
65242,9270263,306,GROCERY,National,BAG SNACKS,POTATO CHIPS,8 OZ
86131,15452032,693,DRUG GM,National,CANDY - CHECKLANE,CANDY BARS (SINGLES)(INCLUDING,1.45 OZ


In [10]:
recommender.get_similar_users_recommendation(user=2374)

[1301100, 6534400, 14050574, 13189726, 15927033]

Модули импортируются корректно. Модель MainRecommender обучается. Методы get_similar_items_recommendation и get_similar_users_recommendation работают корректно.